In [ ]:
import sqlite3

# Create an in-memory SQLite database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()


In [ ]:
# Create sample tables
cursor.execute('''CREATE TABLE employees (
    id INTEGER PRIMARY KEY,
    name TEXT,
    department TEXT,
    salary INTEGER
)''')

cursor.execute('''CREATE TABLE departments (
    id INTEGER PRIMARY KEY,
    name TEXT
)''')

# Insert sample data into employees and departments
departments = [(1, 'HR'), (2, 'Engineering'), (3, 'Sales')]
employees = [
    (1, 'Alice', 'HR', 50000),
    (2, 'Bob', 'Engineering', 70000),
    (3, 'Charlie', 'Sales', 45000),
    (4, 'David', 'Engineering', 80000),
    (5, 'Eve', 'HR', 52000)
]
cursor.executemany('INSERT INTO departments VALUES (?, ?)', departments)
cursor.executemany('INSERT INTO employees VALUES (?, ?, ?, ?)', employees)
conn.commit()


# SQL Window Functions Examples

Below are examples of common SQL window functions, each with a title and description. These examples use the `employees` table created above.

---


## 1. ROW_NUMBER()
Assigns a unique sequential integer to rows within a partition of a result set.

**Query:**
```sql
SELECT id, name, department, salary,
       ROW_NUMBER() OVER (PARTITION BY department ORDER BY salary DESC) AS row_num
FROM employees;
```

---

## 2. RANK()
Assigns a rank to each row within a partition, with gaps for ties.

**Query:**
```sql
SELECT id, name, department, salary,
       RANK() OVER (PARTITION BY department ORDER BY salary DESC) AS rank
FROM employees;
```

---

## 3. DENSE_RANK()
Similar to RANK(), but no gaps in ranking values when there are ties.

**Query:**
```sql
SELECT id, name, department, salary,
       DENSE_RANK() OVER (PARTITION BY department ORDER BY salary DESC) AS dense_rank
FROM employees;
```

---

## 4. NTILE(3)
Divides the result set into 3 approximately equal parts and assigns a bucket number to each row.

**Query:**
```sql
SELECT id, name, department, salary,
       NTILE(3) OVER (PARTITION BY department ORDER BY salary DESC) AS salary_group
FROM employees;
```

---

## 5. LAG()
Accesses data from the previous row in the result set.

**Query:**
```sql
SELECT id, name, department, salary,
       LAG(salary) OVER (PARTITION BY department ORDER BY salary DESC) AS prev_salary
FROM employees;
```

---

## 6. SUM() OVER()
Calculates the cumulative sum of salaries within each department, ordered by salary descending.

**Query:**
```sql
SELECT id, name, department, salary,
       SUM(salary) OVER (PARTITION BY department ORDER BY salary DESC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cumulative_salary
FROM employees;
```


In [ ]:
-- 1. ROW_NUMBER(): Assigns a unique row number within each department, ordered by salary descending
SELECT id, name, department, salary,
       ROW_NUMBER() OVER (PARTITION BY department ORDER BY salary DESC) AS row_num
FROM employees;

-- 2. RANK(): Ranks employees within each department by salary (with gaps for ties)
SELECT id, name, department, salary,
       RANK() OVER (PARTITION BY department ORDER BY salary DESC) AS rank
FROM employees;

-- 3. DENSE_RANK(): Assigns dense ranks within each department by salary (no gaps for ties)
SELECT id, name, department, salary,
       DENSE_RANK() OVER (PARTITION BY department ORDER BY salary DESC) AS dense_rank
FROM employees;

-- 4. NTILE(3): Divides employees into 3 salary groups within each department
SELECT id, name, department, salary,
       NTILE(3) OVER (PARTITION BY department ORDER BY salary DESC) AS salary_group
FROM employees;

-- 5. LAG(): Shows each employee's salary and the previous employee's salary within the same department
SELECT id, name, department, salary,
       LAG(salary) OVER (PARTITION BY department ORDER BY salary DESC) AS prev_salary
FROM employees;

-- 6. SUM() OVER(): Calculates the cumulative sum of salaries within each department
SELECT id, name, department, salary,
       SUM(salary) OVER (PARTITION BY department ORDER BY salary DESC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cumulative_salary
FROM employees;


In [1]:
from pathlib import Path

# Create the content for the .ipynb file as a Python script with markdown and SQL magic
notebook_content = '''{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# SQL Tutorial in Jupyter (SQLite)\n",
    "\n",
    "This notebook demonstrates:\n",
    "- Creating sample tables and inserting data\n",
    "- Using WHERE, GROUP BY, HAVING, ORDER BY\n",
    "- Aggregate Functions\n",
    "- CTEs\n",
    "- Window Functions\n",
    "- Joins (INNER, LEFT, RIGHT simulated, FULL OUTER simulated, SELF)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "%load_ext sql\n",
    "%%sql sqlite://"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Create Sample Tables and Insert Data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "%%sql\n",
    "CREATE TABLE departments (\n",
    "    id INTEGER PRIMARY KEY,\n",
    "    department_name TEXT\n",
    ");\n",
    "\n",
    "CREATE TABLE employees (\n",
    "    id INTEGER PRIMARY KEY,\n",
    "    name TEXT,\n",
    "    salary INTEGER,\n",
    "    department_id INTEGER,\n",
    "    manager_id INTEGER,\n",
    "    status TEXT,\n",
    "    FOREIGN KEY(department_id) REFERENCES departments(id)\n",
    ");\n",
    "\n",
    "INSERT INTO departments VALUES\n",
    "(1, 'HR'),\n",
    "(2, 'Engineering'),\n",
    "(3, 'Sales');\n",
    "\n",
    "INSERT INTO employees VALUES\n",
    "(1, 'Alice', 70000, 2, NULL, 'active'),\n",
    "(2, 'Bob', 48000, 2, 1, 'active'),\n",
    "(3, 'Charlie', 55000, 1, 1, 'active'),\n",
    "(4, 'David', 62000, 3, 1, 'inactive'),\n",
    "(5, 'Eva', 71000, 2, 2, 'active'),\n",
    "(6, 'Frank', 46000, NULL, NULL, 'active');"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## WHERE, GROUP BY, HAVING, ORDER BY"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "%%sql\n",
    "-- Filter before grouping\n",
    "SELECT * FROM employees WHERE salary > 50000;"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "%%sql\n",
    "-- Group by department\n",
    "SELECT department_id, COUNT(*) AS emp_count\n",
    "FROM employees\n",
    "GROUP BY department_id\n",
    "HAVING COUNT(*) > 1;"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Aggregate Functions"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "%%sql\n",
    "SELECT department_id, AVG(salary) AS avg_salary\n",
    "FROM employees\n",
    "GROUP BY department_id\n",
    "ORDER BY avg_salary DESC;"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## CTE (Common Table Expression)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "%%sql\n",
    "WITH dept_avg AS (\n",
    "  SELECT department_id, AVG(salary) AS avg_salary\n",
    "  FROM employees\n",
    "  GROUP BY department_id\n",
    ")\n",
    "SELECT * FROM dept_avg WHERE avg_salary > 60000;"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Window Functions"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "%%sql\n",
    "SELECT *,\n",
    "  ROW_NUMBER() OVER (PARTITION BY department_id ORDER BY salary DESC) AS row_num,\n",
    "  RANK() OVER (PARTITION BY department_id ORDER BY salary DESC) AS rank,\n",
    "  DENSE_RANK() OVER (PARTITION BY department_id ORDER BY salary DESC) AS dense_rank,\n",
    "  LAG(salary, 1) OVER (PARTITION BY department_id ORDER BY salary) AS prev_salary,\n",
    "  LEAD(salary, 1) OVER (PARTITION BY department_id ORDER BY salary) AS next_salary\n",
    "FROM employees;"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Joins"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "%%sql\n",
    "-- INNER JOIN\n",
    "SELECT e.name, d.department_name\n",
    "FROM employees e\n",
    "JOIN departments d ON e.department_id = d.id;"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "%%sql\n",
    "-- LEFT JOIN\n",
    "SELECT e.name, d.department_name\n",
    "FROM employees e\n",
    "LEFT JOIN departments d ON e.department_id = d.id;"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "%%sql\n",
    "-- Simulate FULL OUTER JOIN in SQLite using UNION\n",
    "SELECT e.name, d.department_name\n",
    "FROM employees e\n",
    "LEFT JOIN departments d ON e.department_id = d.id\n",
    "UNION\n",
    "SELECT e.name, d.department_name\n",
    "FROM employees e\n",
    "RIGHT JOIN departments d ON e.department_id = d.id;"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "%%sql\n",
    "-- SELF JOIN to get manager names\n",
    "SELECT e.name AS employee, m.name AS manager\n",
    "FROM employees e\n",
    "LEFT JOIN employees m ON e.manager_id = m.id;"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "name": "python",
   "version": ""
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}
'''

# Save the content to a .ipynb file
notebook_path = Path("/mnt/data/sql_tutorial_with_examples.ipynb")
notebook_path.write_text(notebook_content)

notebook_path.name

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/sql_tutorial_with_examples.ipynb'